In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup

In [2]:
sp500_df = pd.read_csv('sp500_companies.csv')
STOCK_TICKERS = sp500_df['Symbol'].tolist()

START_DATE = '2010-01-01'
END_DATE = '2020-12-31'

MARKET_INDEX_TICKER = '^GSPC'

In [ ]:
FMP_API_KEY = "secret_api_key"
FMP_BASE_URL = "https://financialmodelingprep.com/api/v3"

In [4]:
def pe_pb_ratios(ticker, start_date, end_date):
    endpoint = f"{FMP_BASE_URL}/key-metrics-ttm/{ticker}?apikey={FMP_API_KEY}"
    params = {"apikey": FMP_API_KEY, "limit": 40}

    try:
        response = requests.get(endpoint, params=params)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        data = response.json()

        if not data:
            print(f"No key metrics data found for {ticker} from FMP.")
            return pd.DataFrame()

        # Extract relevant ratios and dates
        ratios_list = []
        for entry in data:
            date_str = entry.get('date')
            if date_str:
                date = pd.to_datetime(date_str)
                # FMP provides 'peRatioTTM' and 'pbRatioTTM'
                pe_ratio = entry.get('peRatioTTM')
                pb_ratio = entry.get('pbRatioTTM')
                ratios_list.append({'date': date, 'PE_Ratio': pe_ratio, 'PB_Ratio': pb_ratio})

        ratios_df = pd.DataFrame(ratios_list).set_index('date').sort_index()

        # Create a full date range and reindex to forward-fill quarterly data to daily
        full_date_range = pd.date_range(start=start_date, end=end_date, freq='D')
        ratios_df = ratios_df.reindex(full_date_range, method='ffill')

        # Drop any rows outside the desired date range if reindex went too far
        ratios_df = ratios_df[(ratios_df.index >= start_date) & (ratios_df.index <= end_date)]

        return ratios_df[['PE_Ratio', 'PB_Ratio']]

    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error for {ticker}: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Error Connecting for {ticker}: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error for {ticker}: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"An unexpected error occurred for {ticker}: {err}")
    except Exception as e:
        print(f"An error processing FMP data for {ticker}: {e}")
    return pd.DataFrame()




In [5]:
def ohlcv_fmp(industry_ticker, start_date, end_date):
    """
    Fetches OHLCV data for an industry ETF from FMP.
    """
    print(f"Fetching industry index OHLCV for {industry_ticker} from FMP...")
    # FMP's historical-price-full endpoint provides OHLCV
    endpoint = f"{FMP_BASE_URL}/historical-price-full/{industry_ticker}?apikey={FMP_API_KEY}"
    params = {"apikey": FMP_API_KEY, "from": start_date, "to": end_date}

    
    try:
        response = requests.get(endpoint, params=params)
        response.raise_for_status()
        data = response.json()

        if not data or 'historical' not in data:
            print(f"No historical data found for industry index {industry_ticker} from FMP.")
            return pd.DataFrame()

        df = pd.DataFrame(data['historical'])
        df['date'] = pd.to_datetime(df['date'])
        df = df.set_index('date').sort_index()
        # Keep only relevant columns and rename if necessary for consistency
        df = df[['open', 'high', 'low', 'close', 'volume']]
        df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
        return df

    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error for industry index {industry_ticker}: {errh}")
        if response.status_code == 403:
            print("Access denied. Check your FMP API key or plan for this endpoint.")
    except requests.exceptions.ConnectionError as errc:
        print(f"Error Connecting for industry index {industry_ticker}: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error for industry index {industry_ticker}: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"An unexpected error occurred for industry index {industry_ticker}: {err}")
    except Exception as e:
        print(f"An error processing FMP industry index data for {industry_ticker}: {e}")
    return pd.DataFrame()

In [6]:
def collect_stock_data(tickers, start, end):
    """
    Collects OHLCV, calculated returns, volume, and fundamental factors
    for a list of stock tickers.
    """
    all_stock_data = {}
    for ticker in tickers:
        print(f"Collecting data for {ticker}...")
        try:
            # 1. Fetch OHLCV data using yfinance (generally robust for common stocks)
            stock_data = yf.download(ticker, start=start, end=end, progress=False)
            if stock_data.empty:
                print(f"No OHLCV data found for {ticker} from yfinance. Skipping.")
                continue

            # 2. Calculate Daily Rate of Return
            stock_data['Return'] = np.log(stock_data['Close'] / stock_data['Close'].shift(1))
            stock_data['Return'].replace([np.inf, -np.inf], np.nan, inplace=True)
            stock_data.dropna(subset=['Return'], inplace=True)

            # 3. Add Volume (already present in yfinance data)
            stock_data['Volume'] = stock_data['Volume']

            # 4. Fetch P/E and P/B Ratios using FMP API
            fundamental_ratios = pe_pb_ratios(ticker, start, end)

            # Align fundamental data with stock_data's index
            # Reindex stock_data to a full date range first, then join and forward fill missing dates in ratios
            # This ensures all dates are covered for ratios if they weren't exactly daily from FMP
            full_date_range = pd.date_range(start=stock_data.index.min(), end=stock_data.index.max(), freq='D')
            stock_data = stock_data.reindex(full_date_range)

            if not fundamental_ratios.empty:
                # Join on index (dates)
                stock_data = stock_data.join(fundamental_ratios[['PE_Ratio', 'PB_Ratio']], how='left')
                # Forward-fill any NaNs in PE/PB that result from daily vs quarterly/sparse FMP data
                stock_data['PE_Ratio'] = stock_data['PE_Ratio'].ffill()
                stock_data['PB_Ratio'] = stock_data['PB_Ratio'].ffill()
            else:
                stock_data['PE_Ratio'] = np.nan # Assign NaN if no data
                stock_data['PB_Ratio'] = np.nan # Assign NaN if no data


            all_stock_data[ticker] = stock_data

        except Exception as e:
            print(f"Error collecting data for {ticker}: {e}")
            continue
    return all_stock_data

In [7]:
def collect_market_index_data(market_index_ticker, start, end):
    """
    Collects OHLCV and calculates rate of return for the market index using yfinance.
    """
    print(f"Collecting market index data for {market_index_ticker}...")
    try:
        market_data = yf.download(market_index_ticker, start=start, end=end, progress=False)
        if market_data.empty:
            print(f"No data found for market index {market_index_ticker}. Skipping.")
            return pd.DataFrame()

        # Calculate Market Index Rate of Return
        market_data['Market_Index_Return'] = np.log(market_data['Close'] / market_data['Close'].shift(1))
        market_data['Market_Index_Return'].replace([np.inf, -np.inf], np.nan, inplace=True)
        market_data.dropna(subset=['Market_Index_Return'], inplace=True)

        return market_data['Market_Index_Return'].rename('Market_Index_Return')
    except Exception as e:
        print(f"Error collecting market index data for {market_index_ticker}: {e}")
        return pd.DataFrame()

In [8]:
def collect_industry_index_data(industry_tickers_map, start, end):
    """
    Collects OHLCV and calculates rate of return for specified industry indices using FMP API.
    industry_tickers_map: A dictionary where keys are stock tickers and values are
                          their corresponding industry index tickers (e.g., ETFs).
    """
    all_industry_returns = {}
    unique_industry_tickers = set(industry_tickers_map.values())

    for industry_ticker in unique_industry_tickers:
        print(f"Collecting industry index data for {industry_ticker} using FMP...")
        industry_data = ohlcv_fmp(industry_ticker, start, end) # Using FMP API call
        if not industry_data.empty:
            industry_data['Industry_Index_Return'] = np.log(industry_data['Close'] / industry_data['Close'].shift(1))
            industry_data['Industry_Index_Return'].replace([np.inf, -np.inf], np.nan, inplace=True)
            industry_data.dropna(subset=['Industry_Index_Return'], inplace=True)
            all_industry_returns[industry_ticker] = industry_data['Industry_Index_Return']
        else:
            print(f"Could not get industry index data for {industry_ticker}.")
    return all_industry_returns

In [9]:
# --- Main Execution ---
if __name__ == "__main__":
    if FMP_API_KEY == "tO4f1GnWKfU2zR4Bt6pm2uFeNTcP9SJW":
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("WARNING: Please replace 'YOUR_API_KEY_HERE' with your actual FMP API key.")
        print("Get your API key from: https://financialmodelingprep.com/developer/docs/")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        # Exit or provide placeholder data if API key is not set
        # For demonstration, we'll continue but FMP calls will fail.
        # exit("API Key not set.")

    if not STOCK_TICKERS:
        print("No valid stock tickers found to process. Please ensure 'sp500_companies.csv' is correctly formatted with 'Symbol' or 'Ticker' column. Exiting.")
    else:
        print(f"Attempting to collect data for {len(STOCK_TICKERS)} stocks from the S&P 500 list.")
        
        # Create a simplified INDUSTRY_TICKERS_MAP for demonstration
        # In a real scenario, this would be much more detailed and accurate,
        # mapping each stock to its specific sector ETF or industry index.
        # Example S&P 500 Sector ETFs: XLK (Technology), XLE (Energy), XLF (Financial), XLV (Health Care)
        # For this example, let's just map everything to XLK for simplicity.
        # You'll need to refine this mapping for your specific stocks.
        INDUSTRY_TICKERS_MAP = {}
        for ticker in STOCK_TICKERS:
            # This is a placeholder. You'd ideally classify your stocks into sectors
            # and map them to the corresponding sector ETF ticker.
            INDUSTRY_TICKERS_MAP[ticker] = 'XLK' # Example: Technology Sector ETF

        # Collect stock-specific data
        # Note: Depending on your FMP plan and number of stocks, this might take a long time
        # or hit rate limits. Consider processing a smaller subset of stocks initially.
        stocks_data = collect_stock_data(STOCK_TICKERS, START_DATE, END_DATE)

        # Collect market index data
        market_index_returns = collect_market_index_data(MARKET_INDEX_TICKER, START_DATE, END_DATE)

        # Collect industry index data
        industry_index_returns = collect_industry_index_data(INDUSTRY_TICKERS_MAP, START_DATE, END_DATE)


        # --- Consolidate Data ---
        final_datasets = {}
        for ticker, df in stocks_data.items():
            if not df.empty:
                # Merge market index returns
                if not market_index_returns.empty:
                    df = df.join(market_index_returns, how='left')

                # Merge industry index returns based on the stock's industry
                stock_industry_ticker = INDUSTRY_TICKERS_MAP.get(ticker)
                if stock_industry_ticker and stock_industry_ticker in industry_index_returns:
                    df = df.join(industry_index_returns[stock_industry_ticker].rename('Industry_Index_Return'), how='left')
                else:
                    print(f"Warning: No industry index data available or mapped for {ticker}. Check INDUSTRY_TICKERS_MAP.")

                # Clean up: forward-fill or drop NaNs for factors (P/E, P/B, index returns)
                # The paper mentions cleaning and preprocessing, including handling missing values.
                # Interpolation is suggested for missing values.
                # Reindex again to ensure all dates from START_DATE to END_DATE are present, and ffill everything
                full_range_for_final = pd.date_range(start=START_DATE, end=END_DATE, freq='D')
                df = df.reindex(full_range_for_final)

                # Forward-fill all columns from the last valid observation
                df.ffill(inplace=True)
                # Then, back-fill any remaining NaNs (e.g., at the beginning if no prior data)
                df.bfill(inplace=True)

                df.dropna(inplace=True) # Drop any rows that still have NaNs (e.g., completely missing data)


                if not df.empty:
                    final_datasets[ticker] = df
                    print(f"\n--- Consolidated Data for {ticker} (First 5 rows) ---")
                    print(df.head())
                    print(f"\n--- Consolidated Data for {ticker} (Last 5 rows) ---")
                    print(df.tail())
                    print(f"\n--- {ticker} Data Info ---")
                    df.info()
                else:
                    print(f"Consolidated data for {ticker} is empty after merging and cleaning.")

        if not final_datasets:
            print("\nNo complete datasets were generated for any stock. This might be due to API key issues, rate limits, or data availability.")
        else:
            print("\nData collection complete. You now have 'final_datasets' dictionary.")
            print("Each key is a stock ticker, and its value is a DataFrame containing:")
            print("- Open, High, Low, Close, Volume (from yfinance)")
            print("- Daily Log Return (Rt)")
            print("- Market Index Rate of Return (from yfinance)")
            print("- Industry Index Rate of Return (from FMP, based on your ETF mapping)")
            print("- Price-Earnings Ratio (PE_Ratio, from FMP)")
            print("- Price-to-Book Ratio (PB_Ratio, from FMP)")
            print("\nRemember to verify data with your FMP API key and consider potential rate limits.")

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Get your API key from: https://financialmodelingprep.com/developer/docs/
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Attempting to collect data for 502 stocks from the S&P 500 list.


C:\Users\PRAJWAL\AppData\Local\Temp\ipykernel_19000\3832743817.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(ticker, start=start, end=end, progress=False)
C:\Users\PRAJWAL\AppData\Local\Temp\ipykernel_19000\3832743817.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data['Return'].replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\PRAJWAL\AppData\Local\Temp\ipykernel_19000\3832743817.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(ticker, start=start, end=end, progress=False)


Error collecting data for AAPL: ['Return']


C:\Users\PRAJWAL\AppData\Local\Temp\ipykernel_19000\3832743817.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data['Return'].replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\PRAJWAL\AppData\Local\Temp\ipykernel_19000\3832743817.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(ticker, start=start, end=end, progress=False)


Error collecting data for NVDA: ['Return']


...

Error collecting data for FMC: ['Return']



1 Failed download:
['AMTM']: YFPricesMissingError('possibly delisted; no price data found  (1d 2010-01-01 -> 2020-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 1262322000, endDate = 1609390800")')
C:\Users\PRAJWAL\AppData\Local\Temp\ipykernel_19000\3706720834.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  market_data = yf.download(market_index_ticker, start=start, end=end, progress=False)


No OHLCV data found for AMTM from yfinance. Skipping.


C:\Users\PRAJWAL\AppData\Local\Temp\ipykernel_19000\3706720834.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_data['Market_Index_Return'].replace([np.inf, -np.inf], np.nan, inplace=True)


Error collecting market index data for ^GSPC: ['Market_Index_Return']
Fetching industry index OHLCV for XLK from FMP...
HTTP Error for industry index XLK: 401 Client Error: Unauthorized for url: https://financialmodelingprep.com/api/v3/historical-price-full/XLK?apikey=tO4f1GnWKfU2zR4Bt6pm2uFeNTcP9SJW&apikey=tO4f1GnWKfU2zR4Bt6pm2uFeNTcP9SJW&from=2010-01-01&to=2020-12-31
Could not get industry index data for XLK.

No complete datasets were generated for any stock. This might be due to API key issues, rate limits, or data availability.


In [4]:
import yfinance as yf
import pandas as pd
import numpy as np

# This is a sample of the tickers from your file
tickers = ['AAPL', 'MSFT', 'AMZN', 'NVDA']
start = '2010-01-01'
end = '2020-12-31'

all_stock_data = {}

for ticker in tickers:
    print(f"Collecting data for {ticker}...")
    try:
        stock_data = yf.download(ticker, start=start, end=end, progress=False)

        # ADD THIS LINE to see what yfinance returns
        print(f"Downloaded data for {ticker}:\n{stock_data.head()}")

        # The rest of your code that handles the data
        stock_data['Return'] = stock_data['Close'].pct_change()
        stock_data['Return'].replace([np.inf, -np.inf], np.nan, inplace=True)
        all_stock_data[ticker] = stock_data['Return']

    except Exception as e:
        print(f"Error collecting data for {ticker}: {e}")

C:\Users\PRAJWAL\AppData\Local\Temp\ipykernel_39580\2762475962.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(ticker, start=start, end=end, progress=False)


Downloaded data for AAPL:
Price          Close      High       Low      Open     Volume
Ticker          AAPL      AAPL      AAPL      AAPL       AAPL
Date                                                         
2010-01-04  6.431896  6.446623  6.382908  6.414465  493729600
2010-01-05  6.443017  6.479382  6.409055  6.449629  601904800
2010-01-06  6.340533  6.468564  6.333921  6.443018  552160000
2010-01-07  6.328810  6.371487  6.282827  6.363974  477131200
2010-01-08  6.370887  6.371489  6.283130  6.320396  447610800


C:\Users\PRAJWAL\AppData\Local\Temp\ipykernel_39580\2762475962.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data['Return'].replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\PRAJWAL\AppData\Local\Temp\ipykernel_39580\2762475962.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(ticker, start=start, end=end, progress=False)


Downloaded data for MSFT:
Price           Close       High        Low       Open    Volume
Ticker           MSFT       MSFT       MSFT       MSFT      MSFT
Date                                                            
2010-01-04  23.211435  23.323930  22.941447  22.963947  38409100
2010-01-05  23.218943  23.323939  22.978954  23.136447  49749600
2010-01-06  23.076445  23.308934  22.888953  23.158940  58182400
2010-01-07  22.836458  23.023950  22.641467  22.971451  50559700
2010-01-08  22.993950  23.158942  22.678964  22.708964  51197400
Downloaded data for AMZN:
Price        Close    High     Low    Open     Volume
Ticker        AMZN    AMZN    AMZN    AMZN       AMZN
Date                                                 
2010-01-04  6.6950  6.8305  6.6570  6.8125  151998000
2010-01-05  6.7345  6.7740  6.5905  6.6715  177038000
2010-01-06  6.6125  6.7365  6.5825  6.7300  143576000
2010-01-07  6.5000  6.6160  6.4400  6.6005  220604000
2010-01-08  6.6760  6.6840  6.4515  6.5280  196610

C:\Users\PRAJWAL\AppData\Local\Temp\ipykernel_39580\2762475962.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data['Return'].replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\PRAJWAL\AppData\Local\Temp\ipykernel_39580\2762475962.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(ticker, start=start, end=end, progress=False)
C:\Users\PRAJWAL\AppData\Local\Temp\ipykernel_39580\2762475962.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data['Return'].replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\PRAJWAL\AppData\Local\Te

Downloaded data for NVDA:
Price          Close      High       Low      Open     Volume
Ticker          NVDA      NVDA      NVDA      NVDA       NVDA
Date                                                         
2010-01-04  0.423854  0.426834  0.415143  0.424313  800204000
2010-01-05  0.430043  0.434628  0.422249  0.422249  728648000
2010-01-06  0.432794  0.433711  0.425688  0.429814  649168000
2010-01-07  0.424313  0.432336  0.421103  0.430502  547792000
2010-01-08  0.425230  0.428210  0.418353  0.420874  478168000


C:\Users\PRAJWAL\AppData\Local\Temp\ipykernel_39580\2762475962.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data['Return'].replace([np.inf, -np.inf], np.nan, inplace=True)


In [6]:
all_stock_data

{'AAPL': Date
 2010-01-04         NaN
 2010-01-05    0.001729
 2010-01-06   -0.015906
 2010-01-07   -0.001849
 2010-01-08    0.006649
                 ...   
 2020-12-23   -0.006976
 2020-12-24    0.007712
 2020-12-28    0.035765
 2020-12-29   -0.013315
 2020-12-30   -0.008527
 Name: Return, Length: 2768, dtype: float64,
 'MSFT': Date
 2010-01-04         NaN
 2010-01-05    0.000323
 2010-01-06   -0.006137
 2010-01-07   -0.010400
 2010-01-08    0.006897
                 ...   
 2020-12-23   -0.013039
 2020-12-24    0.007827
 2020-12-28    0.009922
 2020-12-29   -0.003601
 2020-12-30   -0.011019
 Name: Return, Length: 2768, dtype: float64,
 'AMZN': Date
 2010-01-04         NaN
 2010-01-05    0.005900
 2010-01-06   -0.018116
 2010-01-07   -0.017013
 2010-01-08    0.027077
                 ...   
 2020-12-23   -0.006627
 2020-12-24   -0.003949
 2020-12-28    0.035071
 2020-12-29    0.011584
 2020-12-30   -0.010882
 Name: Return, Length: 2768, dtype: float64,
 'NVDA': Date
 2010-01-04      